# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
df = pd.read_csv('previsao_de_renda.csv').dropna().drop_duplicates()
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('data_ref', axis=1, inplace=True)
df.drop('id_cliente', axis=1, inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12427 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   12427 non-null  object 
 1   posse_de_veiculo       12427 non-null  bool   
 2   posse_de_imovel        12427 non-null  bool   
 3   qtd_filhos             12427 non-null  int64  
 4   tipo_renda             12427 non-null  object 
 5   educacao               12427 non-null  object 
 6   estado_civil           12427 non-null  object 
 7   tipo_residencia        12427 non-null  object 
 8   idade                  12427 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  12427 non-null  float64
 11  renda                  12427 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.1+ MB


1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

##### Parte 1

In [9]:
for col in df.columns:
    if col == 'renda':
        continue
    print(f'{df[col].name}: {df[col].value_counts().index[0]}')

sexo: F
posse_de_veiculo: False
posse_de_imovel: True
qtd_filhos: 0
tipo_renda: Assalariado
educacao: Secundário
estado_civil: Casado
tipo_residencia: Casa
idade: 40
tempo_emprego: 4.216438356164384
qt_pessoas_residencia: 2.0


In [12]:
y_a, X_a = patsy.dmatrices('np.log(renda) ~ '
    'sexo + posse_de_veiculo + '
    'C(posse_de_imovel, Treatment(True)) + '
    'C(qtd_filhos) + '
    'tipo_renda + '
    'C(educacao, Treatment("Secundário")) + '
    'estado_civil + '
    'C(tipo_residencia, Treatment("Casa")) + '
    'idade + '
    'tempo_emprego + '
    'C(qt_pessoas_residencia, Treatment(2.0))', 
data=df)

reg_a = sm.OLS(y_a, X_a).fit()
reg_a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     203.5
Date:                Fri, 23 Sep 2022   Prob (F-statistic):               0.00
Time:                        13:40:33   Log-Likelihood:                -13559.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12392   BIC:                         2.745e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.1759      0.039    185.454      0.000       7.100       7.252
sexo[T.M]                                                       0.7878      0.015     53.664      0.000       0.759       0.817
posse_de_veiculo[T.True]                                        0.0436      0.014      3.079      0.002       0.016       0.071
C(posse_de_imovel, Treatment(True))[T.False]                   -0.0828      0.014     -5.907      0.000      -0.110      -0.055
C(qtd_filhos)[T.1]                                             -0.2501      0.112     -2.239      0.025      -0.469      -0.031
C(qtd_filhos)[T.2]                                             -0.5697      0.223     -2.559      0.011      -1.006      -0.133
C(qtd_filhos)[T.3]                                             -0.5560      0.486     -1.144      0.253      -1.509       0.397
C(qtd_filhos)[T.4]                                              0.2091      0.429      0.488      0.626      -0.631       1.049
C(qtd_filhos)[T.5]                                              0.1385      0.410      0.338      0.736      -0.666       0.943
C(qtd_filhos)[T.7]                                              0.0221      0.162      0.137      0.891      -0.295       0.339
C(qtd_filhos)[T.14]                                            -0.4630      0.261     -1.772      0.076      -0.975       0.049
tipo_renda[T.Bolsista]                                          0.2259      0.241      0.937      0.349      -0.247       0.699
tipo_renda[T.Empresário]                                        0.1547      0.015     10.359      0.000       0.125       0.184
tipo_renda[T.Pensionista]                                      -0.2891      0.243     -1.190      0.234      -0.765       0.187
tipo_renda[T.Servidor público]                                  0.0551      0.022      2.476      0.013       0.011       0.099
C(educacao, Treatment("Secundário"))[T.Primário]                0.0130      0.072      0.181      0.856      -0.128       0.154
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.1168      0.142      0.821      0.412      -0.162       0.396
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1069      0.014      7.678      0.000       0.080       0.134
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0290      0.032     -0.897      0.370      -0.092       0.034
estado_civil[T.Separado]                                        0.3176      0.112      2.831      0.005       0.098       0.537
estado_civil[T.Solteiro]                                        0.2556      0.110      2.319      0.020       0.040       0.472
estado_civi

Há uma quantidade excesssiva de parâmetros. Provavelmente é possível remover alguns.

##### Parte 2

In [26]:
#cat_qt_pessoas = [1.0, 3.0, 4.0, 5.0, 6.0, 7.0, 9.0, 15.0]

y_b, X_b = patsy.dmatrices('np.log(renda) ~ '
    'sexo + '
    'posse_de_veiculo + '
    'C(posse_de_imovel, Treatment(True)) + '
    'C(qtd_filhos) + '
    'tipo_renda + '
    'C(educacao, '
    'Treatment("Secundário")) + '
    'estado_civil + '
    'C(tipo_residencia, Treatment("Casa")) + '
    'idade + '
    'tempo_emprego + '
    'C(qt_pessoas_residencia, Treatment(2.0))', 
data=df)

reg_b = sm.OLS(y_b, X_b).fit()
X_b

DesignMatrix with shape (12427, 38)
  Columns:
    ['Intercept',
     'sexo[T.M]',
     'posse_de_veiculo[T.True]',
     'C(posse_de_imovel, Treatment(True))[T.False]',
     'C(qtd_filhos)[T.1]',
     'C(qtd_filhos)[T.2]',
     'C(qtd_filhos)[T.3]',
     'C(qtd_filhos)[T.4]',
     'C(qtd_filhos)[T.5]',
     'C(qtd_filhos)[T.7]',
     'C(qtd_filhos)[T.14]',
     'tipo_renda[T.Bolsista]',
     'tipo_renda[T.Empresário]',
     'tipo_renda[T.Pensionista]',
     'tipo_renda[T.Servidor público]',
     'C(educacao, Treatment("Secundário"))[T.Primário]',
     'C(educacao, Treatment("Secundário"))[T.Pós graduação]',
     'C(educacao, Treatment("Secundário"))[T.Superior completo]',
     'C(educacao, Treatment("Secundário"))[T.Superior incompleto]',
     'estado_civil[T.Separado]',
     'estado_civil[T.Solteiro]',
     'estado_civil[T.União]',
     'estado_civil[T.Viúvo]',
     'C(tipo_residencia, Treatment("Casa"))[T.Aluguel]',
     'C(tipo_residencia, Treatment("Casa"))[T.Com os pais]',
     'C